In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np

sys.path.insert(0, '../../../batchflow')
sys.path.insert(0, '../../')

from seismiqb import SeismicDataset, HorizonMetrics, SeismicSampler
from batchflow import Pipeline

In [ ]:
GEOMETRY_PATH = ''
HORIZONS_PATH = ''

dataset = SeismicDataset({GEOMETRY_PATH: {'horizons': HORIZONS_PATH}})
print(dataset)

In [ ]:
field = dataset[0]
horizon = field.horizons[0]

In [ ]:
CROP_SHAPE = (1, 128, 256)

sampler = SeismicSampler(labels=dataset.labels, crop_shape=CROP_SHAPE)

sampler.show_sampled()

In [ ]:
BATCH_SIZE = 32

template = (
    Pipeline()
    .make_locations(generator=sampler, batch_size=BATCH_SIZE)
    .load_cubes(dst='images')
    .create_masks(dst='masks')
)
pipeline = template << dataset

In [ ]:
%%time
batch = pipeline.next_batch()
plotter = batch.plot(dilate=[None, 1, [None, 1]])

In [ ]:
CROP_SHAPE = (512, 512, 1)

sampler = SeismicSampler(labels=dataset.labels, crop_shape=CROP_SHAPE, threshold=0., mode='horizon', shift_height=False)

sampler.show_sampled()

In [ ]:
BATCH_SIZE = 32

template = (
    Pipeline()
    .make_locations(generator=sampler, batch_size=BATCH_SIZE)
    .compute_label_attribute(src='horizons:*/amplitudes', dst='images', fill_value=0)
    .compute_label_attribute(src='horizons:*/mask', dst='masks', fill_value=0)
)
pipeline = template << dataset

In [ ]:
%%time
batch = pipeline.next_batch()
plotter = batch.plot(alpha=[1.0, 1.0, [1.0, 0.5]], dilate=[None, 10, None], mask=['<0', None, None])

In [ ]:
n = 5
batch.plot_roll(n=n, components=[['images'], ['images', 'masks']], alpha=[1.0, [1.0, 0.5]] * n)